In [ ]:
import math
import os
import pandas as pd
import numpy as np
import cv2
import numbers
from PIL import Image, ImageEnhance
from matplotlib import image
from matplotlib import pyplot as plt
import colorsys
from scipy.spatial import ConvexHull, Delaunay
import davi
from time import sleep
from tqdm import tqdm
import itertools
from itertools import groupby
from operator import itemgetter
from sklearn.cluster import KMeans


import sklearn
from collections import Counter
from skimage.color import rgb2lab, deltaE_cie76
import random
import hulls

In [ ]:
    
def calculate_new_coords(x_coord, y_coord, angle, distance):
    new_coord_x = x_coord + distance * math.cos(angle)
    new_coord_y = y_coord + distance * math.sin(angle)
    return new_coord_x, new_coord_y

def find_abdomen(x_coord, y_coord, theta ):
    
    length=30
    width=25
    extra=15
    
    coord_list = []
    
    AN0 = calculate_new_coords(x_coord, y_coord, theta+(math.pi/2), length/2) 
    AA0 = calculate_new_coords(AN0[0], AN0[1], theta, width/2) #top left
    AT0 = calculate_new_coords(AN0[0], AN0[1], theta+math.pi, width/2)
    AX0 = calculate_new_coords(AT0[0], AT0[1], theta+math.pi, extra) #bottom left
    
    AN4 = calculate_new_coords(x_coord, y_coord, theta-(math.pi/2), length/2)
    AA4 = calculate_new_coords(AN4[0], AN4[1], theta, width/2) #top right
    AT4 = calculate_new_coords(AN4[0], AN4[1], theta+math.pi, width/2)
    AX4 = calculate_new_coords(AT4[0], AT4[1], theta+math.pi, extra) #bottom right
    
    coord_list.append(AA0)
    coord_list.append(AX0)
    coord_list.append(AA4)
    coord_list.append(AX4)
    
    return coord_list ### top left, bottom left, top right, bottom right
    

## path to data ## 

In [ ]:
path = '/media/rb17990/DATA/EXP1/6ants analysis/DAVI_OUTPUT/duplicates_removed.h5'
data = pd.read_hdf(path)

video_path = '/media/rb17990/DATA/EXP1/6ants/R23-5P-2.mp4'

data

In [ ]:
folder_path = '/home/rb17990/Documents/flags testing/folder_path'

In [ ]:
dataframe_name = os.path.basename(path)
dataframe_name = dataframe_name[:-48]


folder_path = os.path.dirname(path)
folder_path

## find first frame with all 6 detections ##

In [ ]:
abx_coord = []
aby_coord = []
thx_coord = []
thy_coord = []

for i in range(0, len(data)):

    ind1_abx = data[('DLC_dlcrnetms5_full-modelJan10shuffle1_100000', 'ind1', 'abdomen', 'x')][i]
    ind1_thx = data[('DLC_dlcrnetms5_full-modelJan10shuffle1_100000', 'ind1', 'thorax', 'x')][i]    
    
    ind2_abx = data[('DLC_dlcrnetms5_full-modelJan10shuffle1_100000', 'ind2', 'abdomen', 'x')][i]
    ind2_thx = data[('DLC_dlcrnetms5_full-modelJan10shuffle1_100000', 'ind2', 'thorax', 'x')][i]

    ind3_abx = data[('DLC_dlcrnetms5_full-modelJan10shuffle1_100000', 'ind3', 'abdomen', 'x')][i]
    ind3_thx = data[('DLC_dlcrnetms5_full-modelJan10shuffle1_100000', 'ind3', 'thorax', 'x')][i]

    ind4_abx = data[('DLC_dlcrnetms5_full-modelJan10shuffle1_100000', 'ind4', 'abdomen', 'x')][i]
    ind4_thx = data[('DLC_dlcrnetms5_full-modelJan10shuffle1_100000', 'ind4', 'thorax', 'x')][i]

    ind5_abx = data[('DLC_dlcrnetms5_full-modelJan10shuffle1_100000', 'ind5', 'abdomen', 'x')][i]
    ind5_thx = data[('DLC_dlcrnetms5_full-modelJan10shuffle1_100000', 'ind5', 'thorax', 'x')][i]    
    
    ind6_abx = data[('DLC_dlcrnetms5_full-modelJan10shuffle1_100000', 'ind6', 'abdomen', 'x')][i]    
    ind6_thx = data[('DLC_dlcrnetms5_full-modelJan10shuffle1_100000', 'ind6', 'thorax', 'x')][i]    



    ind1_aby = data[('DLC_dlcrnetms5_full-modelJan10shuffle1_100000', 'ind1', 'abdomen', 'y')][i]
    ind1_thy = data[('DLC_dlcrnetms5_full-modelJan10shuffle1_100000', 'ind1', 'thorax', 'y')][i]    
    
    ind2_aby = data[('DLC_dlcrnetms5_full-modelJan10shuffle1_100000', 'ind2', 'abdomen', 'y')][i]
    ind2_thy = data[('DLC_dlcrnetms5_full-modelJan10shuffle1_100000', 'ind2', 'thorax', 'y')][i]

    ind3_aby = data[('DLC_dlcrnetms5_full-modelJan10shuffle1_100000', 'ind3', 'abdomen', 'y')][i]
    ind3_thy = data[('DLC_dlcrnetms5_full-modelJan10shuffle1_100000', 'ind3', 'thorax', 'y')][i]

    ind4_aby = data[('DLC_dlcrnetms5_full-modelJan10shuffle1_100000', 'ind4', 'abdomen', 'y')][i]
    ind4_thy = data[('DLC_dlcrnetms5_full-modelJan10shuffle1_100000', 'ind4', 'thorax', 'y')][i]

    ind5_aby = data[('DLC_dlcrnetms5_full-modelJan10shuffle1_100000', 'ind5', 'abdomen', 'y')][i]
    ind5_thy = data[('DLC_dlcrnetms5_full-modelJan10shuffle1_100000', 'ind5', 'thorax', 'y')][i]    
    
    ind6_aby = data[('DLC_dlcrnetms5_full-modelJan10shuffle1_100000', 'ind6', 'abdomen', 'y')][i]    
    ind6_thy = data[('DLC_dlcrnetms5_full-modelJan10shuffle1_100000', 'ind6', 'thorax', 'y')][i]  

    
    
    if math.isnan(ind1_abx) or math.isnan(ind1_thx):
        continue
    else:
        if math.isnan(ind2_abx) or math.isnan(ind2_thx):
            continue
        else:
            if math.isnan(ind3_abx) or math.isnan(ind3_thx):
                continue
            else:
                if math.isnan(ind4_abx) or math.isnan(ind4_thx):
                    continue
                else:
                    if math.isnan(ind5_abx) or math.isnan(ind5_thx):
                        continue
                    else:
                        if math.isnan(ind6_abx) or math.isnan(ind6_thx):
                            continue
                        else:
                            abx_coord.append([ind1_abx, ind2_abx, ind3_abx, ind4_abx, ind5_abx, ind6_abx])
                            thx_coord.append([ind1_thx, ind2_thx, ind3_thx, ind4_thx, ind5_thx, ind6_thx])
                            aby_coord.append([ind1_aby, ind2_aby, ind3_aby, ind4_aby, ind5_aby, ind6_aby])
                            thy_coord.append([ind1_thy, ind2_thy, ind3_thy, ind4_thy, ind5_thy, ind6_thy])
                            
                            
                            break

print(abx_coord)
print(aby_coord)
print(thx_coord)
print(thy_coord)
print(i)

### one ant missing  

In [ ]:
abx_coord = []
aby_coord = []
thx_coord = []
thy_coord = []

for i in range(0, len(data)):
    
    abx_coord = []
    thx_coord = []


    ind1_abx = data[('DLC_dlcrnetms5_full-modelJan10shuffle1_100000', 'ind1', 'abdomen', 'x')][i]
    ind1_thx = data[('DLC_dlcrnetms5_full-modelJan10shuffle1_100000', 'ind1', 'thorax', 'x')][i]    
    
    ind2_abx = data[('DLC_dlcrnetms5_full-modelJan10shuffle1_100000', 'ind2', 'abdomen', 'x')][i]
    ind2_thx = data[('DLC_dlcrnetms5_full-modelJan10shuffle1_100000', 'ind2', 'thorax', 'x')][i]

    ind3_abx = data[('DLC_dlcrnetms5_full-modelJan10shuffle1_100000', 'ind3', 'abdomen', 'x')][i]
    ind3_thx = data[('DLC_dlcrnetms5_full-modelJan10shuffle1_100000', 'ind3', 'thorax', 'x')][i]

    ind4_abx = data[('DLC_dlcrnetms5_full-modelJan10shuffle1_100000', 'ind4', 'abdomen', 'x')][i]
    ind4_thx = data[('DLC_dlcrnetms5_full-modelJan10shuffle1_100000', 'ind4', 'thorax', 'x')][i]

    ind5_abx = data[('DLC_dlcrnetms5_full-modelJan10shuffle1_100000', 'ind5', 'abdomen', 'x')][i]
    ind5_thx = data[('DLC_dlcrnetms5_full-modelJan10shuffle1_100000', 'ind5', 'thorax', 'x')][i]    
    
    ind6_abx = data[('DLC_dlcrnetms5_full-modelJan10shuffle1_100000', 'ind6', 'abdomen', 'x')][i]    
    ind6_thx = data[('DLC_dlcrnetms5_full-modelJan10shuffle1_100000', 'ind6', 'thorax', 'x')][i]    



    ind1_aby = data[('DLC_dlcrnetms5_full-modelJan10shuffle1_100000', 'ind1', 'abdomen', 'y')][i]
    ind1_thy = data[('DLC_dlcrnetms5_full-modelJan10shuffle1_100000', 'ind1', 'thorax', 'y')][i]    
    
    ind2_aby = data[('DLC_dlcrnetms5_full-modelJan10shuffle1_100000', 'ind2', 'abdomen', 'y')][i]
    ind2_thy = data[('DLC_dlcrnetms5_full-modelJan10shuffle1_100000', 'ind2', 'thorax', 'y')][i]

    ind3_aby = data[('DLC_dlcrnetms5_full-modelJan10shuffle1_100000', 'ind3', 'abdomen', 'y')][i]
    ind3_thy = data[('DLC_dlcrnetms5_full-modelJan10shuffle1_100000', 'ind3', 'thorax', 'y')][i]

    ind4_aby = data[('DLC_dlcrnetms5_full-modelJan10shuffle1_100000', 'ind4', 'abdomen', 'y')][i]
    ind4_thy = data[('DLC_dlcrnetms5_full-modelJan10shuffle1_100000', 'ind4', 'thorax', 'y')][i]

    ind5_aby = data[('DLC_dlcrnetms5_full-modelJan10shuffle1_100000', 'ind5', 'abdomen', 'y')][i]
    ind5_thy = data[('DLC_dlcrnetms5_full-modelJan10shuffle1_100000', 'ind5', 'thorax', 'y')][i]    
    
    ind6_aby = data[('DLC_dlcrnetms5_full-modelJan10shuffle1_100000', 'ind6', 'abdomen', 'y')][i]    
    ind6_thy = data[('DLC_dlcrnetms5_full-modelJan10shuffle1_100000', 'ind6', 'thorax', 'y')][i]  

    abx_coord.append(ind1_abx)
    abx_coord.append(ind2_abx)
    abx_coord.append(ind3_abx)
    abx_coord.append(ind4_abx)
    abx_coord.append(ind5_abx)
    abx_coord.append(ind6_abx)
    thx_coord.append(ind1_thx)
    thx_coord.append(ind2_thx)
    thx_coord.append(ind3_thx)
    thx_coord.append(ind4_thx)
    thx_coord.append(ind5_thx)
    thx_coord.append(ind6_thx)

    nan_check_a = []
    
    for a in abx_coord:
        if math.isnan(a):
            nan_check_a.append(a)
            
    nan_check_t = []
    
    for t in thx_coord:
        if math.isnan(t):
            nan_check_t.append(t)            
            
            
    if len(nan_check_a) == 1 and len(nan_check_t) == 1:
        break
        
    
print(abx_coord)
print(thx_coord)
print(i)

## load first frame ##

In [ ]:
cap = cv2.VideoCapture(video_path)
fps = cap.get(cv2.CAP_PROP_FPS)

folder_path = os.path.dirname(video_path)
form = '.png'

vidname = os.path.basename(video_path)

full_vidname = vidname.strip('.mp4') + '-' + str(i) + form
cap.set(cv2.CAP_PROP_POS_FRAMES, i)
ret, frame = cap.read()

if ret: 
    colour_converted = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)
    
im = Image.fromarray(colour_converted)

enhancer = ImageEnhance.Brightness(im)
factor = 1.5
im_output = enhancer.enhance(factor)
contrast = ImageEnhance.Contrast(im_output)
factor_c = 1.2
im_output_T = contrast.enhance(factor_c)

im_output_T.save(os.path.join(folder_path, full_vidname))
image_path = os.path.join(folder_path, full_vidname)

image = hulls.get_image(image_path)
    

In [ ]:
plt.imshow(image)

## ind1 ##

In [ ]:
theta = davi.get_vector_angle(ind1_abx, ind1_aby, ind1_thx, ind1_thy)
waffle = find_abdomen(ind1_abx, ind1_aby, theta)

top_left_x = int(min([waffle[0][0], waffle[1][0], waffle[2][0], waffle[3][0]]))
top_left_y = int(min([waffle[0][1], waffle[1][1], waffle[2][1], waffle[3][1]]))
bot_right_x = int(max([waffle[0][0], waffle[1][0], waffle[2][0], waffle[3][0]]))
bot_right_y = int(max([waffle[0][1], waffle[1][1], waffle[2][1], waffle[3][1]]))


crop = image[top_left_y : bot_right_y, top_left_x : bot_right_x]


plt.imshow(crop)


## ind2 ##

In [ ]:
theta = davi.get_vector_angle(ind2_abx, ind2_aby, ind2_thx, ind2_thy)
waffle = find_abdomen(ind2_abx, ind2_aby, theta)

top_left_x = int(min([waffle[0][0], waffle[1][0], waffle[2][0], waffle[3][0]]))
top_left_y = int(min([waffle[0][1], waffle[1][1], waffle[2][1], waffle[3][1]]))
bot_right_x = int(max([waffle[0][0], waffle[1][0], waffle[2][0], waffle[3][0]]))
bot_right_y = int(max([waffle[0][1], waffle[1][1], waffle[2][1], waffle[3][1]]))


crop = image[top_left_y : bot_right_y, top_left_x : bot_right_x]


plt.imshow(crop)




## ind3 ##

In [ ]:
theta = davi.get_vector_angle(ind3_abx, ind3_aby, ind3_thx, ind3_thy)
waffle = find_abdomen(ind3_abx, ind3_aby, theta)

top_left_x = int(min([waffle[0][0], waffle[1][0], waffle[2][0], waffle[3][0]]))
top_left_y = int(min([waffle[0][1], waffle[1][1], waffle[2][1], waffle[3][1]]))
bot_right_x = int(max([waffle[0][0], waffle[1][0], waffle[2][0], waffle[3][0]]))
bot_right_y = int(max([waffle[0][1], waffle[1][1], waffle[2][1], waffle[3][1]]))


crop = image[top_left_y : bot_right_y, top_left_x : bot_right_x]


plt.imshow(crop)



## ind4 ##

In [ ]:
theta = davi.get_vector_angle(ind4_abx, ind4_aby, ind4_thx, ind4_thy)
waffle = find_abdomen(ind4_abx, ind4_aby, theta)

top_left_x = int(min([waffle[0][0], waffle[1][0], waffle[2][0], waffle[3][0]]))
top_left_y = int(min([waffle[0][1], waffle[1][1], waffle[2][1], waffle[3][1]]))
bot_right_x = int(max([waffle[0][0], waffle[1][0], waffle[2][0], waffle[3][0]]))
bot_right_y = int(max([waffle[0][1], waffle[1][1], waffle[2][1], waffle[3][1]]))


crop = image[top_left_y : bot_right_y, top_left_x : bot_right_x]


plt.imshow(crop)




## ind5 ##

In [ ]:
theta = davi.get_vector_angle(ind5_abx, ind5_aby, ind5_thx, ind5_thy)
waffle = find_abdomen(ind5_abx, ind5_aby, theta)

top_left_x = int(min([waffle[0][0], waffle[1][0], waffle[2][0], waffle[3][0]]))
top_left_y = int(min([waffle[0][1], waffle[1][1], waffle[2][1], waffle[3][1]]))
bot_right_x = int(max([waffle[0][0], waffle[1][0], waffle[2][0], waffle[3][0]]))
bot_right_y = int(max([waffle[0][1], waffle[1][1], waffle[2][1], waffle[3][1]]))


crop = image[top_left_y : bot_right_y, top_left_x : bot_right_x]


plt.imshow(crop)



## ind6 ##

In [ ]:
theta = davi.get_vector_angle(ind6_abx, ind6_aby, ind6_thx, ind6_thy)
waffle = find_abdomen(ind6_abx, ind6_aby, theta)

top_left_x = int(min([waffle[0][0], waffle[1][0], waffle[2][0], waffle[3][0]]))
top_left_y = int(min([waffle[0][1], waffle[1][1], waffle[2][1], waffle[3][1]]))
bot_right_x = int(max([waffle[0][0], waffle[1][0], waffle[2][0], waffle[3][0]]))
bot_right_y = int(max([waffle[0][1], waffle[1][1], waffle[2][1], waffle[3][1]]))


crop = image[top_left_y : bot_right_y, top_left_x : bot_right_x]


plt.imshow(crop)

